In [32]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
from dotenv import load_dotenv
import pandas as pd
import time
from datetime import datetime, timedelta
import os
import json
load_dotenv()

True

In [10]:
accountID = os.getenv("OANDA_ACCOUNT_ID")
api = API(access_token=os.getenv("OANDA_ACCESS_TOKEN"))
params ={
          "instruments": "USD_JPY"
        }

In [23]:
def process_streaming_response(response, temp_list):
    bid = float(response['closeoutBid'])
    ask = float(response['closeoutAsk'])
    mid = round((bid + ask) / 2, 3)
    temp_list.append(mid)


In [24]:
def get_candlestick_data(timeframe, temp_list):
    open = round(temp_list[0], 3)
    high = round(max(temp_list), 3)
    low = round(min(temp_list), 3)
    close = round(temp_list[-1], 3)
    
    data = {'Open': open, 'High': high, 'Low': low, 'Close': close}
    df = pd.DataFrame(data, index=[timeframe])
    
    return df

In [37]:
# r = pricing.PricingStream(accountID=accountID, params=params)
# temp_list = []
# try:
#     rv = api.request(r)
#     maxrecs = 100
#     for tick in rv:
#         response = json.loads(json.dumps(tick, indent=4))
#         print(response)
#         # price_dict = json.dumps(tick, indent=4)
#         try:
#             process_streaming_response(response, temp_list)
#         except:
#             pass
#         maxrecs -= 1
#         if maxrecs == 0:
#             break
# except oandapyV20.exceptions.V20Error as err:
#     print(r.status_code, err)

In [31]:
interval_start = datetime.now()
interval = timedelta(minutes=10)

In [44]:
def streaming_data_pipeline(accountID, params, api):
    interval_start = datetime.now()
    interval = timedelta(minutes=10)  # Define your interval here
    temp_list = []  # Initialize outside the loop to persist data across iterations

    r = pricing.PricingStream(accountID=accountID, params=params)
    while True:
        try:
            rv = api.request(r)
            for tick in rv:
                try:
                    # Directly use tick since it's already a dict object
                    process_streaming_response(tick, temp_list)
                    print(tick["time"], tick['closeoutBid'], tick['closeoutAsk'])
                    if datetime.now() - interval_start >= interval:
                        interval_start = datetime.now()
                        df = get_candlestick_data(interval_start, temp_list)
                        print(df)
                        temp_list.clear()  # Clear after processing
                except Exception as e:
                    # print(f"Error processing tick: {e}")
                    pass
        except oandapyV20.exceptions.V20Error as err:
            print(f"V20Error encountered: {err}")
            
        

In [45]:
streaming_data_pipeline(accountID, params, api)

2024-04-01T14:40:15.609455066Z 151.739 151.761
2024-04-01T14:40:16.159217342Z 151.739 151.760
2024-04-01T14:40:16.411206460Z 151.741 151.760
2024-04-01T14:40:17.147320193Z 151.740 151.759
2024-04-01T14:40:17.785452319Z 151.731 151.753
2024-04-01T14:40:18.360454099Z 151.734 151.757
2024-04-01T14:40:18.535245516Z 151.738 151.760
2024-04-01T14:40:18.762423191Z 151.738 151.759
Error processing tick: 'closeoutBid'
2024-04-01T14:40:21.341436613Z 151.738 151.760
2024-04-01T14:40:21.624045730Z 151.741 151.760
2024-04-01T14:40:21.902618134Z 151.740 151.758
2024-04-01T14:40:22.917841867Z 151.740 151.757
2024-04-01T14:40:23.295052093Z 151.740 151.758
2024-04-01T14:40:24.709270862Z 151.740 151.757
2024-04-01T14:40:25.744315713Z 151.740 151.758
Error processing tick: 'closeoutBid'
2024-04-01T14:40:26.262664625Z 151.738 151.758
2024-04-01T14:40:27.308146165Z 151.738 151.757
2024-04-01T14:40:27.802171091Z 151.740 151.757
2024-04-01T14:40:28.781544416Z 151.738 151.757
2024-04-01T14:40:29.036770925Z 15

KeyboardInterrupt: 